In [40]:
import re

from google.cloud import storage
from io import BytesIO

import pandas as pd

In [2]:
client = storage.Client()

In [5]:
bucket = client.get_bucket('ames-house-dataset')

In [55]:
def retrieve_file(file_name, gs_bucket):
    blob = storage.Blob(file_name, gs_bucket)
    content = blob.download_as_string()
    return content

def display_description(field_name, description):
    print(re.search(re.escape(field_name) + r':.+\n\n(.+\n)+', description)[0])

In [33]:
train = pd.read_csv(BytesIO(retrieve_file('train.csv', bucket)))

In [56]:
desc = retrieve_file('data_description.txt', bucket).decode('utf-8')